# Behavioral Cloning


In [5]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import csv
import cv2
#import keras
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam
from time import time
%matplotlib inline
print('Import successful.')

Import successful.


Load the data and get an impression of the images

In [6]:
source_path = 'data/driving_log.csv'

logs = []
reader = csv.reader(open(source_path, 'rt'))
for line in reader:
    logs.append(line)

print('logs created succesfully')

logs created succesfully


In [7]:
print('logs.pop[0]: ', logs.pop(0))
print('len(logs[0]): ', len(logs[0]))
print('logs[0]: ', logs[0])
print('logs[1][0]: ', logs[1][0])

logs.pop[0]:  ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']
len(logs[0]):  7
logs[0]:  ['IMG/center_2016_12_01_13_30_48_287.jpg', ' IMG/left_2016_12_01_13_30_48_287.jpg', ' IMG/right_2016_12_01_13_30_48_287.jpg', ' 0', ' 0', ' 0', ' 22.14829']
logs[1][0]:  IMG/center_2016_12_01_13_30_48_404.jpg


Define necessary functions

In [8]:
def resize_image(input_image):
    '''@brief Reduce image size for computational reasons.
    '''
    x_size = 48
    y_size = 128
    return cv2.resize(input_image, (y_size, x_size))

def crop_image(input_image):
    '''@brief Cut off the sky and lower part of the picture since it contains
              little information about the track.
    '''
    offset_low = 40
    offset_high = 140
    return input_image[offset_low:offset_high]

def change_color_space(input_image):
    ''' Return image in S dimension of HSV color space
    '''
    return  cv2.cvtColor(input_image, cv2.COLOR_RGB2HSV)[:, :, 1]

def image_wrapper(input_image):
    '''@brief summarize all relevant image processing functions
    '''
    #return change_color_space(resize_image(crop_image(input_image)))
    return resize_image(crop_image(input_image))

def img_load(path_to_table, steering_offset):
    ''' @brief Load the images from an input path and perform 
               particular preprocessing operations on them.'''
    #read the path to the images
    reader = csv.reader(open(source_path, 'rt'))
    new_logs = []
    for line in reader:
        new_logs.append(line) #save the path in a list
    #delete first line because it contains unnecessary information
    new_logs.pop(0)
    #print('logs[0]: ', new_logs[0]) #debug
    X = []
    y = []
    for center, left, right, steering, throttle, brake, speed in new_logs:
        #print('center: ', center)
#        try: 
            # preprocess the images and save them in a list
            #X.append(image_wrapper(plt.imread('data/'+center))) # include preprocessing
            #X.append(cv2.imread('data/'+center)) #exclude preprocessing
#        except FileNotFoundError: #debug
#            print('center: ', center) #debug
#            print('left: ', left) #debug
#            print('right: ', right) #debug
#            print('len(X_train): ', len(X)) #debug
#            print('logs[len(X_train)-1]: ', new_logs[len(X)-1]) #debug
        #append left and right images    
        #X.append(image_wrapper(plt.imread('data/'+center))) # include preprocessing
        #X.append(image_wrapper(plt.imread('data/'+left[1:]))) # include preprocessing
        #X.append(image_wrapper(plt.imread('data/'+right[1:]))) # include preprocessing
        
        X.append(plt.imread('data/'+center)) # exclude preprocessing
        X.append(plt.imread('data/'+left[1:])) # exclude preprocessing
        X.append(plt.imread('data/'+right[1:])) # exclude preprocessing
        
        #X.append(cv2.imread('data/'+left[1:]))
        #X.append(cv2.imread('data/'+right[1:]))
        
        #get the steering angle
        y.append(float(steering))
        #add the steering offset to the left and right images
        y.append(float(steering) + steering_offset)
        y.append(float(steering) - steering_offset)

    return X, y

Load data into appropriate lists

In [14]:
t_start = time()
X_train, y_train = img_load('data/driving_log.csv', 0.08)
print('img_load took: ', (time() - t_start) /60, ' s minutes to load.')
print('len(X_train): ', len(X_train)) #debug
print('len(X_train[0]): ', len(X_train[0])) #debug
print('len(X_train[0][0]): ', len(X_train[0][0])) #debug
print('len(X_train[0][0][0]): ', len(X_train[0][0][0])) #debug
print('len(y_train): ', len(y_train)) #debug

img_load took:  3.2801823218663535  s minutes to load.
len(X_train):  48441
len(X_train[0]):  48
len(X_train[0][0]):  128
len(X_train[0][0][0]):  3
len(y_train):  48441


Flip images around the y-axis

In [15]:
#flip the images around the y-axis
X_flipped = []
y_flipped = []
for X in X_train:
    X_flipped.append(np.fliplr(X))
# adjust the steering by changing the foresign
for y in y_train:
    y_flipped.append(-y)
    
# merge flipped data with original data
X_total = X_train + X_flipped
y_total = y_train + y_flipped

print('len(X_total): ', len(X_total))
print('len(y_total): ', len(y_total))
#print('X_total.shape: ', X_total.shape)

len(X_total):  96882
len(y_total):  96882


In [16]:
X_np = np.array(X_total)
y_np = np.array(y_total)
print('X_np.shape: ', X_np.shape)
print('y_np.shape: ', y_np.shape)

X_np.shape:  (96882, 48, 128, 3)
y_np.shape:  (96882,)


In [17]:
# TODO: Shuffle the data
X_np, y_np = shuffle(X_np, y_np) 
image_size = np.shape(np.array(X_np[0]))
print('image_size: ', image_size)
#print('X_np[0]: ', X_np[0]) #debug

image_size:  (48, 128, 3)
X_np[0]:  [[[132 131  99]
  [128 127  97]
  [138 134 109]
  ..., 
  [188 225 255]
  [188 225 255]
  [188 225 255]]

 [[147 148 116]
  [126 125  96]
  [116 114  88]
  ..., 
  [193 229 254]
  [193 229 254]
  [193 229 254]]

 [[151 154 122]
  [126 128  98]
  [111 110  83]
  ..., 
  [198 234 255]
  [198 234 255]
  [198 234 255]]

 ..., 
 [[110 104  87]
  [109 103  86]
  [146 140 122]
  ..., 
  [175 171 118]
  [158 155 115]
  [167 161 134]]

 [[211 204 170]
  [209 202 168]
  [209 202 167]
  ..., 
  [179 176 127]
  [202 199 153]
  [159 155 116]]

 [[224 211 169]
  [224 212 169]
  [215 203 160]
  ..., 
  [122 118  86]
  [142 140 100]
  [192 192 149]]]


Now that the image preproccessing finished, the neural net needs to be defined in Keras. 

In [18]:
model = Sequential()
#normalize image depending on the input size of the image
if X_np[0].shape[0] == 160:
    model.add(Lambda(lambda x: x/255.0 - 0.5,input_shape=(160,320,3)))
    #cut off the sky and hood in the camera images
    model.add(Cropping2D(cropping=((70,25),(1,1))))
elif X_np[0].shape[0] == 48:
    model.add(Lambda(lambda x: x/255.0 - 0.5,input_shape=(48,128,3)))
else:
    raise TypeError() #raise a random error

#reduce computational load for faster performance
#actual neural net layout starts here
model.add(Convolution2D(24, 12, 64, activation='relu'))
#model.add(MaxPooling2D()) #deal with overfitting
model.add(Convolution2D(34, 6, 32, activation='relu'))
#model.add(MaxPooling2D())
model.add(Convolution2D(48, 4, 11,subsample=(2,2),activation='relu'))
model.add(Convolution2D(64, 2, 3,activation='relu'))
model.add(Convolution2D(64, 2, 3,activation='relu'))
model.add(Flatten())
model.add(Dense(1164))     
model.add(Dense(100)) 
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

#Display overview of the network
model.summary()
# Network architecture
#model = Sequential()
#model.add(Flatten(input_shape = (12, 32)))
#model.add(Dense(1))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 48, 128, 3)    0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 37, 65, 24)    55320       lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 34, 34)    156706      convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)  (None, 15, 12, 48)    71856       convolution2d_2[0][0]            
___________________________________________________________________________________________

In [ ]:
from time import time
start_time = time()
# train the model
# keras model compile, choose optimizer and loss func
model.compile(optimizer='adam',loss='mse')
model.fit(X_np, y_np, validation_split=0.2, shuffle= True, nb_epoch=5)
model.save('model.h5')
print('Total training time: ', time() - start_time)
print('Model created successfully.')

Train on 77505 samples, validate on 19377 samples
Epoch 1/5
   64/77505 [..............................] - ETA: 27056s - loss: 117.1931